# Noise and correlation

In this notebook we'll explore some properties of noise and cross-correlated time series, as most radio-astronomical electric field measurements are in practice indistinguishable from normally distributed noise.

## Setup
First (if you haven't opend that window yet) open contextual help bij pressing "Ctrl+i".

We now begin with a fairly standard first cell to enable automatic reloading of external modules when those external modules are changed (useful if one develops a Python library in conjunction with the notebook).

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab widget

The next step is to include the astropy units framework, which is a fantastic way to prevent (or catch) silly mistakes in formulas.

In [ ]:
import astropy.units as u

The next cell contains a useful utility to easily make figures. Carefully read it to see what is does. If necessary refer to the matplotlib documentation.

In [ ]:
def hdfig(subplots_def=None, scale=0.5):
    fig = plt.figure(figsize=(8,4.5), dpi=scale*1920/8)
    if subplots_def is None:
        return fig
    else:
        return fig, fig.subplots(*subplots_def) # What does the asterisk here mean?

## Noise generation

It is now time to generate some Gaussian noise! The functions to generate it can be found in the `ifrsim/noise.py`  file. Open this file and study the functions `normalized_gaussian_noise` and `digitize`. Note the long documentation strings and included examples. These examples can actually be run and verified using the `pytest` program, so they can double as light-weight unit tests of the software.

First we include the module:

In [ ]:
from ifrsim import noise

Complete the following two cells. Use 4 bit digitisation and assume Nyquist sampling of the real-valued voltages at the antenna terminals. If there are any concepts you do not grasp yet in the previous sentence, refer to the book(s) and study them first. Ensure `num_samp` becomes an integer of value 2000000. If you have trouble with unit conversions, refer to the astropy units documentation and see if one of the "dimensionless" units can be of any use. 

Before generating the simulated data, we first define the structure of the problem: we need to generate 10 seconds of data with a bandwidth of 1 MHz.

In [ ]:
delta_t         = 10*u.s      # Integration time
delta_freq      = 1*u.MHz     # Bandwidth
sample_interval =   # Interval between individual samples. Convert to an appropriate unit FILL_IN
num_samples     =   # FILL_IN
print(num_samples, '|', sample_interval)

The following cell should generate the "original" noise samples as floating point numbers, after which they should be digitized using a simulated 4-bit analogue to digital converter (ADC).

In [ ]:
unit_noise_float = noise.normalized_gaussian_noise( )  # FILL_IN
unit_noise_dig   = noise.digitize( ) # FILL_IN
print(unit_noise_float[:10])
print(unit_noise_dig[:10])

Verify the above output to see if it did what you would expect. Let's see what that looks like, but first study the following cell and ensure you understand every bit of code in detail. We define a function for this plot to make inspecting similar data further on easier.

In [ ]:
def plot_sampled_signal(original_signal, sampled_signal, sample_interval, caption=None, decimation=10000):
    x_time = np.arange(original_signal.shape[0])*sample_interval
    fig , (ax_time, ax_hist) = hdfig((2,1))
    fig.subplots_adjust(hspace=0.3)

    ax_time.plot(x_time[::decimation], original_signal[::decimation], alpha=0.5, label='Original')
    ax_time.scatter(x_time[::decimation], sampled_signal[::decimation], c='red', s=10, alpha=0.2, label='Sampled')
    ax_time.legend()
    ax_time.set_xlabel('Time [%s]' % x_time.unit)
    ax_time.set_ylabel('Value')

    if caption:
        ax_time.set_title(caption)

    ax_hist.hist(original_signal, bins=201, range=(-10.05,10.05), label='Original', log=True)
    ax_hist.hist(sampled_signal  , bins=201, range=(-10.05,10.05), label='Sampled')
    ax_hist.set_xlabel('Value')
    ax_hist.set_ylabel('Count')
    ax_hist.legend()

In [ ]:
plot_sampled_signal(unit_noise_float, unit_noise_dig, sample_interval, caption='Gaussian noise')

**QUESTIONS** regarding the above plot:

1. What are the most extreme values that the "Sampled" data can assume?

2. Do you see those in the plots?

3. Explain why that is good or bad.


## Cross-correlation power and weak signals

Astronomical signals are typically extremely weak: easily thousands to even millions of times smaller in power density at the antenna terminals than the system noise. Let's see how that would work. The total power detector in a modern radio telescope is usually a correlator that computes the autocorrelation (in case of a single antenna) or cross-correlation product (in case of a baseline consisting of two antennas). Analogous to a couple of cells earlier, define three Gaussian signals:

1. The thermal noise for antenna 1: unit standard deviation
2. The thermal noise for antenna 2: unit standard deviation
3. The gaussian signal from the universe: a standard deviation corresponding to a power density a factor 500 smaller than the noise power density for any of the antennas

We use the same integration time and bandwidth as before.

In [ ]:
instantaneous_power_SNR_ratio = 1/500.0
instantaneous_voltage_SNR_ratio =    # FILL_IN

ant1_noise_float =   # FILL_IN
ant2_noise_float =   # FILL_IN
sky_signal_float =   # FILL_IN

Let's first look at the samples (4 bits) of these signals individually:

In [ ]:
nrbits = 4
ant1_sampled = noise.digitize(   ) # FILL_IN
ant2_sampled = noise.digitize(   ) # FILL_IN
sky_sampled  = noise.digitize(   ) # FILL_IN

In [ ]:
plot_sampled_signal(ant1_noise_float, ant1_sampled, sample_interval, caption='Ant 1')

In [ ]:
plot_sampled_signal(ant2_noise_float, ant2_sampled, sample_interval, caption='Ant 2')

In [ ]:
plot_sampled_signal(sky_signal_float, sky_sampled, sample_interval, caption='Sky')

**QUESTION**:
Describe what you see in the above three plots, particularly the last one.

**Extra**
Play with the nrbits and the amplitudes (standard deviation) of the noise and signals in code cells 10--13 and see what happens.

How CAN we detect a signal when all its sampled values are 0? Let's first write a total power detector for two time series. Read up on `scipy`'s `signal.correlate` function to ensure you understand *exactly* what it does.

In [ ]:
from scipy import signal

def simple_real_cross_power(ant_1_voltage, ant_2_voltage, nrbits:int=4):
    assert  ant_1_voltage.shape == ant_2_voltage.shape
    s1 = noise.digitize(   ) # FILL_IN. Make sure to pass on the required number of bits for the ADC
    s2 = noise.digitize(   ) # FILL_IN
    return  signal.correlate(    , mode='same')/s1.shape[0] # FILL_IN


def plot_real_xc(xc:'np.ndarray[np.float]', width:int, sample_interval, caption=None):
    fig, ax = hdfig((1,1))
    m  = xc.shape[0]//2
    lw = width//2
    hw = width - lw
    delay = np.arange(-lw, hw)*sample_interval
    ax.plot(delay, xc[m-lw:m+hw])
    ax.set_xlabel('Delay [%s]' % delay.unit)
    if caption:
        ax.set_title(caption)

We now calculate the autocorrelation of the noise of antenna 1. This may take several seconds up to a minute.

In [ ]:
xc_1_1 = simple_real_cross_power(   ) #FILL_IN

In [ ]:
plot_real_xc(xc_1_1, width=1000, sample_interval=sample_interval, caption='Antenna 1 noise autocorrelation')

Inspect the plot (zoom in if necessary)

**QUESTIONS**:

1. Why is there only a single point with a high value? And why is that peak at exactly 0?

2. What is its value?

3. What is the standard deviation of the off-peak signal? (you can use the empty cell below to calculate. If there is no empty cell, press "ESC" then "b" to create one below. Use "a" instead" of "b" to create one above.

4. Explain why the standard deviation of the off-peak area in the plot has that value?

Oddly, the peak is not at 1.0. Let's investigate the effect of quantisation:

5. Change nrbits to 16 and try again: what happens with the peak and standard deviation?

6. Now multiply the individual `antN_noise_float` parameters by 16 to much better sample the signal. Make sure to divide the result of `simple_real_cross_power` by an appropriate amount to maintain normalization. What happens with the peak and standard deviation?


Now let's do that for the cross correlation of the noise between antenna 1 and 2, EXCLUDING the sky signal:

In [ ]:
xc_1_2 = simple_real_cross_power(  ) #FILL_IN

In [ ]:
plot_real_xc(xc_1_2, width=1000, sample_interval=sample_interval, caption='Antenna 1-2 noise crosscorrelation')

**QUESTIONS**

Again inspect the figure.

1. Why is there NO central peak?

2. What is the standard deviation of the cross correlation function?

3. Explain why it has that value


Now let's add the sky signal to the antenna's noise values before sampling:

In [ ]:
ant1_noise_sky0 = ant1_noise_float +   # FILL_IN
ant2_noise_sky0 =                      # FILL_IN
xc_1_2_noise_with_sky = simple_real_cross_power(ant1_noise_sky0, ant2_noise_sky0)

In [ ]:
plot_real_xc(xc_1_2_noise_with_sky, width=1000, sample_interval=sample_interval, caption='Antenna 1-2 crosscorrelation noise+sky')

Again, carefully inspect the above figure and answer the following questions:

1. What causes the central peak?

2. Explain why the central peak is at 0 delay.

3. What is its amplitude?

4. Explain the value of the central peak's amplitude.

5. What is the standard deviation of the off-peak noise?

6. Considering the previous component-wise sampled signals (noise and sky separate), how is it possible that there is a central peak at all?


Now add a delay of approximately 5 µs to the sky signal at antenna 2. That is: the sky signal arrives approximately 5 µs LATER at antenna 2 than at antenna 1. Hints: 
1. use careful array indexing to get the job done
2. it is not a problem if the data sets are somewhat truncated.

In [ ]:
sample_shift =   # FILL_IN integer number of samples to shift the sky signal between antenna 1 and 2. 
print(sample_shift)
s = sample_shift

ant1_noise_sky = ant1_noise_float[  ] + sky_signal_float[  ]  # FILL_IN, make use of s
ant2_noise_sky = ant2_noise_float[  ] + sky_signal_float[  ]  # FILL_IN, make use of s
xc_1_2_noise_with_sky_delayed = simple_real_cross_power(ant1_noise_sky, ant2_noise_sky)

In [ ]:
plot_real_xc(xc_1_2_noise_with_sky_delayed, width=1000,
             sample_interval=sample_interval,
             caption='Antenna 1-2 crosscorrelation noise+sky delayed')

Inspect the plot. What does "Delay" actually mean here?
1. How much *earlier* the signal arrives at antenna 1
2. How much *later* the signal arrives at antenna 1


## Complex correlator

To speed up many calculations in radio astronomy, it is easier to work with a complex representation of the electric field, where the complex voltage at frequency $\nu$ can be written as
\begin{equation}
E(\nu, t) = a \mathrm{e}^{2\pi\mathrm{i}\nu t}
\end{equation}
and
\begin{equation}
a = |a|\mathrm{e}^{\mathrm{i}\phi_0}
\end{equation}
is the complex amplitude and phase offset of the (quasi)-monochromatic wave at $t=0$.

Using Euler's rule, we can write the complex voltage as
\begin{equation}
E(\nu, t) = a \left(\cos{2\pi\nu t}  + \mathrm{i}\sin{2\pi\nu t}\right)
\end{equation}

Modern radio receivers generally give one the real and imaginary parts of the complex field straight away. The real part is then called the "in-phase" part ($I$), and the imaginary part the "quadrature-phase" part ($Q$) because in the monochromatic case it is 90 degrees out of phase. In radio-engineering this is called "IQ" data. In radio-astronomy we typically call it "complex voltages".

These parts can be obtained by heterodyne-mixing (multiplying) the incoming field with a cosine wave of frequency $\nu$ for the real part, and a corresponding sine wave for the imaginary part.


**QUESTIONS**

1. Explain why the sine wave must be $-sin$

Assuming a Nyquist-sampled, real-valued time series of samples $s_k$, with a total band width $\Delta\nu$, show that when mixed with (co)sine waves of frequency $\nu_0 = \frac{1}{2}\Delta\nu$,

2. $\cos(2\pi\nu_0t)$ at sample $k$ can be written as 1, 0, -1, 0, 1, 0, … for $k$ starting at 0.

3. $\sin(2\pi\nu_0t)$ at sample $k$ can be written as 0,1, 0, -1, 0, 1, … for $k$ starting at 0.


Using smart numpy array indexing, implement such a real-to complex conversion:

In [ ]:
def make_complex_nyquist_timeseries(real_timeseries):
    n           = real_timeseries.shape[0]//2   # Number of complex samples
    cos_samples = real_timeseries[   ] # FILL_IN
    sin_samples = real_timeseries[   ] # FILL_IN
    wave = (-1)**np.arange(n)
    return (cos_samples - 1j*sin_samples)   # Use "wave" somehow to obtain correct result


#TEST CASE to see if you did this correctly
n = 10
real_series = noise.normalized_gaussian_noise(n,seed=1)
print('Real   :', real_series)
complex_series = make_complex_nyquist_timeseries(real_series)
print('Complex:', complex_series)
test_series = np.array([ 0.34558419-0.82161814j, -0.33043708-1.30315723j,  0.90535587-0.44637457j,
  0.53695324+0.5811181j,   0.3645724 -0.2941325j ])
if np.linalg.norm(complex_series - test_series) < 1e-6:
    print('SUCCESS! You did it!')
else:
    print('Not there yet...')    

**QUESTION:** How is the fact that there are only half a number of complex samples at intervals of $\Delta t = 1/\Delta\nu$ not in violation of the Nyquist sampling theorem?


## Forming narrow channels

The math describing radio-interferometric imaging is only valid for the quasi-monochromatic case. Therefore, an actual correlator always involves a channelisation step. Either *before* correlation (FX for "Fourier transform *then* multiply") or after correlation ("XF" for multiply *then* Fourier transform). In case of a large amount of antennas FX costs less compute power.

Let's make an "FX"  correlating interferometer simulation for a single baseline. The simulator should:

1. Take each real time series and digitize it using `nrbits`
2. Convert both time series to complex voltages (IQ data)
3. Partition the long sequences into blocks that can be channelized using an FFT (use `np.fft.ifft`)
4. Multiply the spectrum of antenna 1 with the complex conjugate of the spectrum of antenna 2 (EXPLAIN WHY THIS IS THE SAME AS CROSS-CORRELATING TIME-SERIES)
5. Average the result
6. return the result

Now read the documentation for `np.fft.ifft` and `np.fft.ifftshift`. Use a single call to the latter to ensure that the centre of the band ends up at position `num_chan//2`.

In [ ]:
def fx_correlating_interferometer(signal_1_float, signal_2_float, num_chan:int, nrbits:int=8):
    # "cv" stands for "complex voltage"
    cv_1 =  # FILL_IN complex voltage
    cv_2 =  # FILL_IN
    num_samples = min(len(cv_1), len(cv_2))
    result = np.zeros(num_chan, dtype=np.complex64)
    spectra_added = 0
    for i in range(0, num_samples, num_chan):
        if i + num_chan > num_samples: # can't make a full spectrum anymore
            break
        spectrum_1 = np.fft.ifft(   )    # FILL_IN (normalization "ortho", see np.fft documentation)
        spectrum_2 = np.fft.ifft(   )    # FILL_IN
        result        +=                 # FILL_IN
        spectra_added += 1
    if spectra_added > 0:
        return  # FILL_IN: make use of "spectra_added"
    else:
        raise ValueError("ERROR: Cannot form %a channels from sampled data of lengths %r and %r",
                         num_chan, len(signal_1_float), len(signal_2_float))

We slightly increase the signal-to-noise ratio to ensure we can do the following simulations with only 10 seconds of data. Create a version of `ant1_noise_sky` and `ant2_noise_sky` with three times the `sky_signal_float` voltages at a sample delay `s`.

In [ ]:
ant1_strong =   # FILL_IN
ant2_strong =   # FILL_IN

spectrum = fx_correlating_interferometer(ant1_strong, ant2_strong,
                                        num_chan=1024, nrbits=12)

In [ ]:
def plot_complex_seq(z, xaxis=None):
    fig, (ax_abs, ax_phase) = hdfig((2,1))
    if xaxis is None:
        xlabel, xvalues = 'Channel', np.arange(len(z))
    else:
        xlabel, xvalues = xaxis
        xlabel += ' [%s]' % xvalues.unit
    ax_abs.plot(xvalues, np.absolute(z))
    ax_abs.set_ylabel('Amplitude')
    ax_phase.set_xlabel(xlabel)
    ax_phase.set_ylabel('Phase [rad]')
    ax_phase.plot(xvalues, np.angle(z))

In [ ]:
plot_complex_seq(spectrum)
print(9/500, 3*sky_signal_float.std(), np.absolute(spectrum).mean())

Verify that the mean amplitude is independent of the number of channels you use (try 128...1024 in powers of 2)

**QUESTIONS**

1. Why is the mean amplitude approximately *twice* 9/500?

2. Why is there a phase gradient?

3. Why does it wrap the number of times you see in the plot?


# Verifying frequency

We now add a "calibration" signal to the real valued time-series data in the form of a cosine wave at a frequency of $\frac{1}{3}\Delta\nu$. Give it an amplitude of 0.02: again well below the digitizer's level separation of 1.

In [ ]:
freq_cal       = delta_freq/3
time_of_sample = delta_t*np.arange(num_samples))
narrow_band = 0.02*np.cos(   ) # FILL_IN
narrow_band = narrow_band.value
print(freq_cal)

# nsc = noise + 3x sky shifted by s samples + calibration
ant1_nsc =   # FILL_IN
ant2_nsc =   # FILL_IN


In [ ]:
num_chan=4096
spectrum_cal = fx_correlating_interferometer(ant1_nsc, ant2_nsc,
                                        num_chan=num_chan, nrbits=8)

In [ ]:
plot_complex_seq(spectrum_cal)

Modify the plot command in the previous cell such that the horizontal axis is the frequency instead of the channel number. You've done it correctly if the frequency of the test signal matches.

Experiment with `num_chan` again.

**QUESTIONS**

1. What happens with the calibration signal amplitude?

2. What happens with the sky signal amplitude?

3. Explain the difference


Now let's plot the Fourier transform of the spectrum of the cross-correlation. Because... why not?

In [ ]:
plot_complex_seq(np.fft.fftshift(np.fft.fft(spectrum_cal)))

**QUESTIONS**

1. What does the horizontal axis represent?

2. What causes the single peak?

3. Explain why the single peak is at exactly the place you find it.


## Finally

You've made it until the end of this tutorial. Hopefully you now have a better grasp of the inner workings of the digital signal chain of a radio interferometer.